In [1]:
import numpy as np
import pandas as pd
import locale
import json
import re
import pymorphy2
import torch
import accelerate
from datetime import datetime
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from bs4 import BeautifulSoup
from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import T5Tokenizer
from transformers import T5ForConditionalGeneration, Trainer, TrainingArguments
from peft import get_peft_model, LoraConfig

/Users/alexanderknyshov/Desktop/LLM/Data/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Loading JSON-dataset to dictionary
with open('/Users/alexanderknyshov/Desktop/LLM/Data/datasets/train_set.json', 'r') as json_file:
    data = json.load(json_file)

In [3]:
def format_text(text):
    news_text = "form a recommendation using news: " + " ".join(text['news'])
    recommendation_text = " ".join(text['recommendations'])
    return {'input_text': news_text, 'target_text': recommendation_text}

# Formatting original dataset
dataset = Dataset.from_dict(data)
dataset = dataset.map(format_text)
# Removing unneccessery columns
dataset = dataset.remove_columns(['news', 'recommendations'])

Map: 100%|██████████| 5177/5177 [00:02<00:00, 1979.81 examples/s]


In [4]:
# Splitting into train and validate data
split_ratio = 0.8
split_index = int(len(data['news']) * split_ratio)

train_news = data['news'][:split_index]
train_recommendations = data['recommendations'][:split_index]
train_data = {'news': train_news, 'recommendations': train_recommendations}

val_news = data['news'][split_index:]
val_recommendations = data['recommendations'][split_index:]
val_data = {'news': val_news, 'recommendations': val_recommendations}

# Returning to the format needed
train_dataset = Dataset.from_dict(train_data)
train_dataset = train_dataset.map(format_text)
train_dataset = train_dataset.remove_columns(['news', 'recommendations'])

val_dataset = Dataset.from_dict(val_data)
val_dataset = val_dataset.map(format_text)
val_dataset = val_dataset.remove_columns(['news', 'recommendations'])

Map: 100%|██████████| 1036/1036 [00:00<00:00, 2813.20 examples/s]


In [5]:
# Loading the tokenizer
tokenizer = T5Tokenizer.from_pretrained('sberbank-ai/ruT5-base')

# Method for data tokenization
def tokenize_data(example):
    input_encodings = tokenizer(example['input_text'], padding='max_length', truncation=True, max_length=512)
    target_encodings = tokenizer(example['target_text'], padding='max_length', truncation=True, max_length=256)

    example['input_ids'] = input_encodings['input_ids']
    example['attention_mask'] = input_encodings['attention_mask']
    example['labels'] = target_encodings['input_ids']
    return example

# Train and validation data tokenization
tokenized_train_dataset = train_dataset.map(tokenize_data)
tokenized_val_dataset = val_dataset.map(tokenize_data)

# Getting rid of text columns
tokenized_train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
tokenized_val_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/Users/alexanderknyshov/Desktop/LLM/Data/venv/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Map: 100%|██████████| 1036/1036 [00:01<00:00, 534.99 examples/s]


In [6]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

In [7]:
lora_config = LoraConfig(
    r=16,                       # Ранг, может быть изменен в зависимости от задачи
    lora_alpha=32,             # Гиперпараметр, который можно настроить
    lora_dropout=0.1,          # Вероятность дропаута для LoRA
    target_modules=["q", "v"]  # Модули, которые будут адаптированы
)

In [8]:
# Model Loading
model = T5ForConditionalGeneration.from_pretrained('sberbank-ai/ruT5-base').to(device)
model = get_peft_model(model, lora_config)

# Defining training params
training_args = TrainingArguments(
    output_dir= '/Users/alexanderknyshov/Desktop/LLM/Data/model',
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=1,
    evaluation_strategy="epoch",
    save_strategy="epoch",   
    logging_dir='/Users/alexanderknyshov/Desktop/LLM/Data/model/logs',
    logging_steps=10,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model='eval_loss'
)

# Metrics count
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    loss = sum([1 if pred == label else 0 for pred, label in zip(decoded_preds, decoded_labels)]) / len(decoded_preds)
    return {'accuracy': loss}

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/Users/alexanderknyshov/Desktop/LLM/Data/venv/lib/python3.11/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [9]:
trainer.train()

# Evaluation
eval_results = trainer.evaluate()

# Results
print(f"Validation results: {eval_results}")

  0%|          | 10/4141 [00:04<22:29,  3.06it/s] 

{'loss': 13.2212, 'grad_norm': 9.34701919555664, 'learning_rate': 4.987925621830476e-05, 'epoch': 0.0}


  0%|          | 20/4141 [00:07<21:31,  3.19it/s]

{'loss': 11.9242, 'grad_norm': 12.915538787841797, 'learning_rate': 4.975851243660952e-05, 'epoch': 0.0}


  1%|          | 30/4141 [00:11<23:27,  2.92it/s]

...

{'loss': 6.3944, 'grad_norm': 3.7604923248291016, 'learning_rate': 4.239314175319971e-05, 'epoch': 0.15}


 15%|█▌        | 640/4141 [03:24<19:34,  2.98it/s]

{'loss': 6.2128, 'grad_norm': 4.235511779785156, 'learning_rate': 4.227239797150447e-05, 'epoch': 0.15}


 16%|█▌        | 643/4141 [03:25<19:27,  3.00it/s]

KeyboardInterrupt: 